In [1]:
import torch
print(torch.__version__)

2.4.0+cpu


## Перцептрон в PyTorch

$x = (x_1, ..., x_n)$ — вектор с данными.

$y = (y_1, ..., y_k)$ — то, что хотим предсказать, используя $x$.

Сначала попробуем на совсем простой модели:

$\hat{y} = \sigma ( x^T \cdot W + b)$

$$
W =
\begin{pmatrix}
w_{1,1} & w_{1,2} & ... & w_{1,k}\\
...&...&...&...\\
w_{n,1} & w_{n,2} & ... & w_{n,k}\\
\end{pmatrix}
- \text{Матрица весов}\quad ; \quad b =
\begin{pmatrix}
b_{1} & b_{2} & ... & b_{k}\\
\end{pmatrix}
- \text{Вектор смещения}
$$

$$
x^T \cdot W + b =
\begin{pmatrix}
b_1 + \sum_{i=1}^{n} x_i \cdot w_{i,1} & b_2 + \sum_{i=1}^{n} x_i \cdot w_{i,2} & ... & b_k + \sum_{i=1}^{n} x_i \cdot w_{i,k}\\
\end{pmatrix}
$$

$\sigma(a) = \frac{1}{1 + e^{-a}}$ — функция «сигмоида», в нашем случае применяется поэлементно (к каждому элементу вектора по отдельности).

In [2]:
# Пусть есть вектор x и истинный ответ y_true.
x = torch.rand(3)
y_true = torch.tensor([1.])
print(f"x:\n{x}\n\ny_true:\n{y_true}")

x:
tensor([0.4225, 0.9686, 0.6591])

y_true:
tensor([1.])


In [3]:
# Зададим матрицу весов и вектор смещения.
# С помощью параметра requires_grad мы указываем,
#  что для данных тензоров нужно будет считать градиенты.
w = torch.randn(3, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(f"w:\n{w}\n\nb:\n{b}")

w:
tensor([[-1.1853],
        [ 0.7312],
        [-0.7299]], requires_grad=True)

b:
tensor([0.], requires_grad=True)


In [10]:
# Для вычисления сигмоиды возьмём уже готовую функцию из библиотеки PyTorch.
torch.sigmoid(torch.tensor([1, 0, -1]))

tensor([0.7311, 0.5000, 0.2689])

In [11]:
# Вычисляем наше предсказание на основе простой модели.
y_pred = torch.sigmoid(x.T @ w + b)
y_pred

tensor([0.4320], grad_fn=<SigmoidBackward0>)

In [12]:
# Знак транспонирования при умножении вектора на матрицу можно не писать.
y_pred = torch.sigmoid(x @ w + b)
y_pred

tensor([0.4320], grad_fn=<SigmoidBackward0>)

In [13]:
# Будем оценивать модуль разницы между оригинальным значением y_true и предсказанным y_pred
#  (в более общем случае сумму по всем k, но в нашем примере k = 1).
loss = ((y_true - y_pred)**2).sum()
loss

tensor(0.3226, grad_fn=<SumBackward0>)

In [14]:
# По сути в переменной loss находится число — величина ошибки.
# Именно это число мы хотим *оптимизировать* при обучении:
#  чем оно меньше после очередной итерации, тем лучше наше предсказание
# Будем называть эту переменную «Величина ошибки обучения» или просто «Ошибка».

# Всё, что нужно для завершения шага обучения нашей простой модели — обновить матрицу весов 
#  и вектор смещения, используя соответствующие градиенты (алгоритм градиентного спуска).

# Получить градиенты для любой переменной можно с помощью параметра .grad,
#  однако изначально они ничем не инициализированы.
print(
    f"До применения backward",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

# Функция backward(), используя граф вычислений, дойдёт по нему до всех переменных
#  с requires_grad = True и посчитает для них градиенты.
# Воспользуемся ей для расчёта градиентов относительно ошибки предсказания.
loss.backward()

print(
    f"После применения backward",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
)

До применения backward

w.grad:
None

b.grad:
None

---------

После применения backward

w.grad:
tensor([[-0.1178],
        [-0.2700],
        [-0.1837]])

b.grad:
tensor([-0.2787])


In [15]:
# Обновим наши веса в соответствии с полученными градиентами.
# Пока не используем параметр «шаг градиентного спуска».
print(
    f"До обновления",
    f"w:\n{w}",
    f"b:\n{b}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

with torch.no_grad():
    # Специальная обёртка, чтобы указать, что операции внутри
    #  не требуют построения графа вычислений или же подсчёта градиентов.
    
    # Важно, что пишем w -= w.grad, а не w = w - w.grad.
    # Иначе мы не меняем текущую переменную, а создаём новую,
    #  из-за чего слетят настройки requires_grad = True.
    w -= w.grad  
    b -= b.grad  

print(
    f"После обновления",
    f"w:\n{w}",
    f"b:\n{b}",
    sep="\n\n",
)

До обновления

w:
tensor([[-1.1853],
        [ 0.7312],
        [-0.7299]], requires_grad=True)

b:
tensor([0.], requires_grad=True)

---------

После обновления

w:
tensor([[-1.0675],
        [ 1.0012],
        [-0.5462]], requires_grad=True)

b:
tensor([0.2787], requires_grad=True)


In [16]:
# ОЧЕНЬ ВАЖНО - градиенты не обнуляются, это нужно делать ВРУЧНУЮ.

print(
    f"До обнуления",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

w.grad.zero_()  # In-place обнуление тензора.
b.grad.zero_()

print(
    f"После обнуления",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
)

До обнуления

w.grad:
tensor([[-0.1178],
        [-0.2700],
        [-0.1837]])

b.grad:
tensor([-0.2787])

---------

После обнуления

w.grad:
tensor([[0.],
        [0.],
        [0.]])

b.grad:
tensor([0.])
